Notwendige Bibliotheken:

pip install pvlib tabulate

In [1]:
import sys
#!{sys.executable} -m pip install pvlib tabulate

Hier gibt es nichts zu sehen :-)

Bitte ausführen und runter scrollen.

In [2]:
import csv
import tabulate
from pvlib import iotools 
import time
import json
import os
import pandas as pd 

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [3]:
def loadSvensLastProfil(filename):
    lastprofil = []
    with open(filename) as csvdatei:
        csv_reader_object = csv.reader(csvdatei, delimiter=',')
        for row in csv_reader_object:
            row.pop(0)
            lastprofil.append([float(x) for x in row]) 
    assert len(lastprofil) % 24 == 0, f"Die Anzahl der Einträge muss durch 24 teilbar sein. Nur komplette Tage sind zulässig: {len(lastprofile)}" 
    days = int(len(lastprofil) / 24)

    tmp = chunkIt(lastprofil, days)
    lastprofil = []
    for l in tmp:
        out = []
        for v in l:
            out.extend(v)
        assert len(out) == 1440, f"Falsche Arraylänge: {len(out)}" 
        lastprofil.append(out)

    # Willkürliches Auffüllen des Lastprofiles mit bereis vorhanden Daten, bis ein Jahr voll ist
    pos = 0
    while (len(lastprofil) < 365):
        lastprofil.append(lastprofil[pos % days])
        pos += 1
    assert len(lastprofil) == 365, f"Aufbau des Lastprofiles: 365 Einträge mit jeweils 1440 Einträgen: {len(lastprofil)}"            

    jahresVerbrauch = sum([sum(v) for v in lastprofil])
    print("Stromverbrauch pro Jahr: " + str(round(jahresVerbrauch/1000)))
    return lastprofil


In [4]:
def loadPVGISDataFromCVSFile(filename, year):
    pvgis = []
    with open(filename) as csvdatei:
        csv_reader_object = csv.reader(csvdatei, delimiter=',')
        for row in csv_reader_object:
            if len(row) == 0 or not row[0].startswith(str(year)):
                continue
            pvgis.append(row)
    assert len(pvgis) == 8760, f"8760 (24 Einträge pro Tag * 365 Tage)Einträge für ein Jahr erwartet: {len(pvgis)}" 
    
    # Umwandlung der Stundenwerten in rechnerische Minutenwerte
    out = []
    for tagesdaten in chunkIt(pvgis, 365):
        tag = []
        for stundeninfo in tagesdaten:
                tag.extend([float(stundeninfo[1])/60.0 for x in range(60)])
        out.append(tag)
    assert len(out) == 365, f"365 Einträge mit jeweils 1440 Einträgen: {len(out)}"            
    # Summe für Prüfung bilden
    summe = sum([ sum(x) for x in out ])
    print("PV Ertrag im Jahr:" + str(round(summe / 1000)))
    return out


def convertPVGISData(data, year):
    pvgis = []
    for index, row in data.iterrows():
        if (str(index).startswith(str(year))):
            pvgis.append(row['P'])
    assert len(pvgis) == 8760, f"8760 (24 Einträge pro Tag * 365 Tage)Einträge für ein Jahr erwartet: {len(pvgis)}" 

    # Umwandlung der Stundenwerten in rechnerische Minutenwerte
    out = []
    for tagesdaten in chunkIt(pvgis, 365):
        tag = []
        for stundeninfo in tagesdaten:
             tag.extend([float(stundeninfo)/60.0 for x in range(60)])
        out.append(tag)
    assert len(out) == 365, f"365 Einträge mit jeweils 1440 Einträgen: {len(out)}"   
    return out

def getFilename(d, year):
    filename="pvgis-"
    for x in sorted(d):
        filename += x + str(d[x])
    filename += str(year) + ".data"
    return filename    

def getPVGISData(cfg, year):
    # PV Daten von PVGIS abrufen
    pvdata = [  ]
    for pv in cfg["pv"]:
        # Check for Cache file
        filename = getFilename(pv, year)
        if (os.path.isfile(filename)):
            print("Cached: " + filename)
            data = pd.read_pickle(filename)
            data = convertPVGISData(data, year)
        else:
            print("Request from pvgis: " + filename)
            param = dict(pv)
            param["start"] = year
            param["end"] = year
            rawdata = iotools.get_pvgis_hourly(**param)
            rawdata[0].to_pickle(filename)
            data = convertPVGISData(rawdata[0], year)
                  
        pvdata.append(data)
        time.sleep(3.1)

    # Aufsummieren
    pvErtrag = [[0]*1440 for x in range(0,365)]
    for pv in pvdata:
        for tagIdx in range(0,365):
            for minIdx in range(0,1440):
                pvErtrag[tagIdx][minIdx] += pv[tagIdx][minIdx]


    # Summe für Prüfung bilden
    summe = sum([ sum(x) for x in pvErtrag ])
    print("PV Ertrag im Jahr:" + str(round(summe / 1000)))
    return pvErtrag
    

In [5]:

    
def data2speech(v):
    out = ""
    out += "Netzbezug, " if v["netz"] > 0 else ""
    out += "Einspeisung, " if v["netz"] < 0 else ""
    out += "Ladung, " if v["battery"] > 0 else ""
    out += "Entladung, " if v["battery"] < 0 else ""
    return out

def stats(data,cfg):
    v = {"verbrauch": 0.0, "pv":0.0 , "netzbezug":0.0,"netzeinspeisung": 0.0, "batterieladung":0.0, "batterieendladung": 0.0}
    for tage in data:
        for i in tage:
            v["verbrauch"] += i["verbrauch"]
            v["pv"] += i["pv"]
            if (i["netz"] > 0):
                v["netzbezug"] += i["netz"]
            else:
                v["netzeinspeisung"] -= i["netz"]

            if (i["battery"] > 0):
                v["batterieladung"] += i["battery"]
            else:
                v["batterieendladung"] -= i["battery"]
    for key in  v:
        v[key] = round(v[key]/1000)
    v["kapazität"] = cfg["battery"]["capacity"]
    v["vollzyklen"] = 0 if cfg["battery"]["capacity"] == 0 else round(v["batterieendladung"] / (cfg["battery"]["capacity"]/1000))
    return v
 

Die eigentliche Routine zur Simulation:
* pvErtrag 2D-Array(365x1440) mit dem pvErtrag für jeden Tag des Jahres pro Minate 
* lastprofile 2D-Array(365x1440) mit dem Verbrauch für jeden Tag des Jahres pro Minute
* cfg Config-Objekt

In [6]:
def simulate(pvErtrag, lastprofil, cfg):
    assert len(pvErtrag) == 365, f"365 Einträge mit jeweils 1440 Einträgen: {len(pvErtrag)}"            
    assert len(lastprofil) == 365, f"365 Einträge mit jeweils 1440 Einträgen: {len(lastprofil)}"
    for i in range(0,365):
        assert len(pvErtrag[i]) == 1440, f"365 Einträge mit jeweils 1440 Einträgen: {len(pvErtrag[i])}"            
        assert len(lastprofil[i]) == 1440, f"365 Einträge mit jeweils 1440 Einträgen: {len(lastprofil[i])}"            
    
    data = {
        "battery": {
            "level": 0
        }
    }
    jahresDaten = []
    for tagIdx in range(0,365):
        tagesData = []
        pv = pvErtrag[tagIdx]
        verbrauch = lastprofil[tagIdx]
        for minIdx in range(0,1440):
            v = { "stunde": minIdx // 60, "verbrauch": verbrauch[minIdx], "pv": pv[minIdx], "netz": 0, "battery": 0, "batterylevel": 0}

            # Differenz aus Verbrauch und PV-Ertrag
            balance = v["verbrauch"] - v["pv"]

            # Batterie Ladung/Enladugn bestimmen
            if (balance <= 0):
                # PV Produktion ist höher -- Strom verfügbar
                v["battery"] = min(cfg["battery"]["capacity"] - data["battery"]["level"], -balance, cfg["battery"]["chargelimit"])
            elif (balance > 0):
                # Stromverbrauch ist höher -- Strom wird benötigt
                v["battery"] = -min(data["battery"]["level"], balance, cfg["battery"]["discargelimit"])

            # Auf Basis der Battery Ladung/Enladung Werte anpassen
            data["battery"]["level"] += v["battery"]
            v["netz"] = balance + v["battery"]
            v["batterylevel"]= data["battery"]["level"]

            v["desc"] = data2speech(v)
            tagesData.append(v)
        jahresDaten.append(tagesData)
    return jahresDaten

Ab hier geht es richtig los:
* Konfiguration
* Lastprofil laden
* PV Ertrag erstellen
* Ausgabe des Gesamtertrages (zum prüfen)


In [7]:
    
cfg = {
    "battery": {
        "capacity": 3000,
        "chargelimit": 3000/60,
        "discargelimit": 3000/60,
        
    },
    "pv": [
        { "latitude":52.51890653973749, 
          "longitude":13.375925533235622, 
          "surface_tilt":30,
          "surface_azimuth":-90, 
          "loss":10,
          "pvcalculation":True, 
          "peakpower":9.8
        },
        {
            "latitude":52.51890653973749, 
            "longitude":13.375925533235622, 
            "surface_tilt":30, 
            "surface_azimuth":90, 
            "loss":10,
            "pvcalculation":True, 
            "peakpower":9.8
        }
    ]
}

Genutzt wird ein fiktives Lastprofil. Aus dem Lastprofile für einen Tag, wir ein Lastprofil für 365 Tage erstellt.

Die Funktion getPVGISData lädt die Information automatisch von pvgis herunter. (kann bis zu einer Minute dauern)
Die Daten werden zwischengespeichert und den Server zu schonen.

Teilweise liefert der pvgis-Server einen Timeout zurück. In diesem Fall es später (>3 min) nochmal probieren.


In [8]:
last = loadSvensLastProfil('lastprofile_eintag.csv')
# Option 1 => Direkt vom pvgis Server
pvErtrag = getPVGISData(cfg, 2015)

# Opeion 2=> von einer heruntergeladenen CSV-Datei von pvgis
#pvErtrag = loadPVGISDataFromCVSFile("Timeseries_51.XXX_6.XXX_SA2_8kWp_crystSi_10_30deg_90deg_2005_2020.csv",2005)

Stromverbrauch pro Jahr: 5133
Cached: pvgis-latitude52.51890653973749longitude13.375925533235622loss10peakpower9.8pvcalculationTruesurface_azimuth-90surface_tilt302015.data
Cached: pvgis-latitude52.51890653973749longitude13.375925533235622loss10peakpower9.8pvcalculationTruesurface_azimuth90surface_tilt302015.data
PV Ertrag im Jahr:17153


In [9]:
# Einfachster Fall
# Simulation von einem Fall
simulation = simulate(pvErtrag, last, cfg)
summary = stats(simulation,cfg)
tabulate.tabulate([summary], tablefmt='html', headers="keys")

verbrauch,pv,netzbezug,netzeinspeisung,batterieladung,batterieendladung,kapazität,vollzyklen
5133,17153,1437,13458,1305,1305,3000,435


In [10]:
# Komplexer Fall
# Simulation von verschiedenen Akku Größen
import copy
simulation = []
for i in range(0,20):
    currentCfg = copy.deepcopy(cfg)
    currentCfg["battery"]["capacity"] = i * 1000
    currentCfg["battery"]["chargelimit"] = 2000 / 60
    currentCfg["battery"]["discargelimit"] = 2000 / 60
    j = simulate(pvErtrag, last, currentCfg)
    
    summary = stats(j,currentCfg)
    
    summary["maxPreis"] = round(summary["batterieladung"] * 0.248 *10) # 10 Jahre, 0.248 Ersparnis pro kWh
    simulation.append(summary)

In [11]:
tabulate.tabulate(simulation, tablefmt='html', headers="keys")

verbrauch,pv,netzbezug,netzeinspeisung,batterieladung,batterieendladung,kapazität,vollzyklen,maxPreis
5133,17153,2742,14763,0,0,0,0,0
5133,17153,2093,14114,650,650,1000,650,1612
5133,17153,1747,13768,995,995,2000,498,2468
5133,17153,1537,13558,1205,1205,3000,402,2988
5133,17153,1393,13413,1350,1349,4000,337,3348
5133,17153,1302,13321,1443,1441,5000,288,3579
5133,17153,1241,13259,1504,1502,6000,250,3730
5133,17153,1211,13228,1536,1532,7000,219,3809
5133,17153,1193,13209,1554,1550,8000,194,3854
5133,17153,1179,13194,1570,1564,9000,174,3894
